<a href="https://colab.research.google.com/github/geekevgin/-Python/blob/main/BelEvSer_NLP_HW_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [3]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 48.3 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=80e1e7b32c70764ecc6586c11b9654992c57546149709999018601d26406d95e
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [5]:
import pandas as pd
import pickle
import pandas as pd
import numpy as np
import string
import numpy as np
from tqdm import tqdm_notebook
import re
from pymorphy2 import MorphAnalyzer
import json
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.models import Word2Vec, FastText

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
import gensim.downloader as api

In [7]:
api.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [8]:
word_vectors = api.load("glove-wiki-gigaword-100")

[=================================================-] 98.1% 125.7/128.1MB downloaded


In [9]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 29.6 MB/s eta 0:00:00


In [10]:
from datasets import load_dataset 
dataset = load_dataset('IlyaGusev/gazeta', revision = "v2.0")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60964 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6793 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6369 [00:00<?, ? examples/s]

Dataset gazeta downloaded and prepared to /root/.cache/huggingface/datasets/IlyaGusev___gazeta/default/2.0.0/c329f0fc1c22ab6e43e0045ee659d0d43c647492baa2a6ab3a5ea7dac98cd552. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
for split, data in dataset.items():
  data.to_csv(f"my-dataset.csv", index = None)

Creating CSV from Arrow format:   0%|          | 0/61 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

In [12]:
data = pd.read_csv('my-dataset.csv')

In [13]:
data = data[['text', 'title']]
data

,text,title
0,"В 2020 году инфляция в России составит 3,5-4%,...",Дорогой 2020-й: какие продукты подскочат в цене
1,Глава Белого дома Дональд Трамп выразил надежд...,Подарок от Ким Чен Ына: Трамп ответил на новые...
2,Председатель Союза еврейских религиозных общин...,Это манипуляция: раввин Польши ответил Путину ...
3,Первой песней наступившего 2020 года на Первом...,Первая после Путина: Ротару выступила на Новый...
4,"Выражение «глубинное государство», пришедшее и...","Пресли, Леннон, Трумэн: против кого работал Эд..."
...,...,...
6364,"33% опрошенных россиян убеждены, что дети школ...","Треть россиян считают, что школьники должны по..."
6365,Глава белорусских католиков митрополит Тадеуш ...,Приехавшего из Польши главу католиков Белорусс...
6366,Турецкая делегация направилась в Москву для пе...,Турция отправила в Москву делегацию для обсужд...
6367,Для желающих приобрести инвестиционные продукт...,Банки могут начать тестировать клиентов на зна...


In [14]:
!pip install --upgrade razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
!rm -f gazeta_raw.txt
!wget https://www.dropbox.com/s/4fxj5wmt7tjr5f2/gazeta_raw.txt
!rm -f gazeta_raw_new.jsonl
!wget https://www.dropbox.com/s/gyv841mph04qweu/gazeta_raw_new.jsonl

--2023-04-23 18:11:20--  https://www.dropbox.com/s/4fxj5wmt7tjr5f2/gazeta_raw.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/4fxj5wmt7tjr5f2/gazeta_raw.txt [following]
--2023-04-23 18:11:21--  https://www.dropbox.com/s/raw/4fxj5wmt7tjr5f2/gazeta_raw.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2ce874f4b67e73af086ef9736f.dl.dropboxusercontent.com/cd/0/inline/B6tGDkIjkhLr8q4YQM4HL5xyvx1ssjlhpiuE_hn4MW_Y4orWfFXaD2NkX1a0J55fMOHJIyt_YbcarDTWOwX5NXeLMukTKjm9eQbKT5RB-bw7wYEJHxclC7B3ZhukFFzomk3zfQl6kVCLrDaO7JN2dUuglU3H734ZqGfAyWBWQV_KoQ/file# [following]
--2023-04-23 18:11:21--  https://uc2ce874f4b67e73af086ef9736f.dl.dropboxusercontent.com/cd/0/inline/B6tGDkIjkhLr8q4YQM4HL5xyvx1ssjlhpiuE_hn4MW_Y4orWfFXaD2NkX1a0J55fMOHJIyt_Yb

In [16]:
records = []
with open("gazeta_raw.txt", "r") as r:
    for line in r:
        records.append(json.loads(line))
with open("gazeta_raw_new.jsonl", "r") as r:
    for line in r:
        records.append(json.loads(line))
records.sort(key=lambda x: x["date"])

In [17]:
assert True

#Small preprocess of the answers

question = None
written = False

with open("prepared_answers.txt", "w") as fout:
    with open("gazeta_raw.txt", "r") as r:
        for line in tqdm_notebook(r):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

<ipython-input-17-f705ef850398>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(r):


0it [00:00, ?it/s]

In [18]:
counter_all = 0
counter_filter = 0
with open("prepared_answers.txt", "r") as r:
    for line in tqdm_notebook(r):
        counter_all +=1
        spls = line.split("\t")
        if len(spls[0].split()) < 2 or len(spls[1].split()) < 3 or len(spls[0].split()) > 15:
            continue
            
        counter_filter +=1

<ipython-input-18-6ce9c17a3bd3>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(r):


0it [00:00, ?it/s]

In [19]:
!head -5 prepared_answers.txt

{"url": "https://www.gazeta.ru/auto/2005/06/09_a_298154.shtml", "text": "Вот что я вам скажу о Мценске. На улице Машиностроителей стоит палатка с фруктами, а в палатке торгует девица лет двадцати пяти, в синем халате. Медлительная, как облако. У нее на левой груди выколота роза размером почти с кулак и надписью: «Мой милый Андерсен». Зрелище для самых стойких ценителей женской красоты. Рядом на той же улице передвижная цирковая касса. Маленькая, полукруглая, с окном как в скворечнике. Именно в такой кассе Буратино пытался купить билет в театр Карабаса. А лесковской романтики, впрочем, как тургеневской и бунинской, в городе не осталось. Одно название. Поэтому проехали мы Мценск в печали. И быстро. Возле поселка Тельчье пытались подманить лошадь, привязанную на обочине, чтобы она голову в «Мазду» просунула для фото. Но лошадь оказалась умной и скромной. Фотографировать давала только задницу и хвост, коим отмахивалась от мух. В первом часу пополудни мы обогнали еще одну лошадку, запряженн

In [20]:
stop_words = set(stopwords.words('russian'))

In [21]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in stop_words and i != ""]
    return spls

In [22]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
stop_words = set(stopwords.words('russian'))
exclude = set(string.punctuation)
c = 0

with open("gazeta_raw.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

<ipython-input-22-24b370f22e31>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [23]:
sentences = [i for i in sentences if len(i) > 2]

In [24]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=1)

In [ ]:
modelFT = FastText(sentences=sentences, vector_size=300, min_count=1, window=5, workers=8)

In [ ]:
!pip install annoy

In [ ]:
import annoy

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 100000:
            break

w2v_index.build(10)
ft_index.build(10)

In [ ]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3, )
    return [index_map[i] for i in answers]

In [ ]:
TEXT = "Александр Лукашенко"

In [ ]:
TEXT_1 = "Чемпионат мира затронул каждого"

In [ ]:
TEXT_3 = "Работники завода устроили забастовку"

In [ ]:
TEXT_2 = "Средняя заработная плата в столице"


In [ ]:
get_response(TEXT, w2v_index, modelW2V, index_map)

In [ ]:
get_response(TEXT_1, w2v_index, modelW2V, index_map)

In [ ]:
get_response(TEXT_2, w2v_index, modelW2V, index_map)

In [ ]:
get_response(TEXT_3, w2v_index, modelW2V, index_map)

In [ ]:
get_response(TEXT, ft_index, modelFT, index_map)

In [ ]:
get_response(TEXT_1, ft_index, modelFT, index_map)

In [ ]:
get_response(TEXT_2, ft_index, modelFT, index_map)

In [ ]:
get_response(TEXT_3, ft_index, modelFT, index_map)